In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 905.3/905.3 kB 19.3 MB/s eta 0:00:00


In [2]:
import torch
import os
import glob
from PIL import Image
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
# Example usage:
BKAI_train_images_folder = '/kaggle/input/bkai-dataset2/vietnamese/train_images'
BKAI_labels_folder = '/kaggle/input/bkai-dataset2/vietnamese/labels'

BKAI_val_images_folder = '/kaggle/input/bkai-dataset2/vietnamese/test_image'

BKAI_test_images_folder = '/kaggle/input/bkai-dataset2/vietnamese/unseen_test_images'

BKAI_train_output_images_folder = "/kaggle/working/train/images"
BKAI_train_output_labels_folder = "/kaggle/working/train/labels"

BKAI_val_output_images_folder = "/kaggle/working/val/images"
BKAI_val_output_labels_folder = "/kaggle/working/val/labels"

BKAI_test_output_images_folder = "/kaggle/working/test/images"
BKAI_test_output_labels_folder = "/kaggle/working/test/labels"

# Define paths
BKAI1_train_images_folder = '/kaggle/input/bkai-dataset-1/train_imgs/training_img'
BKAI1_train_labels_folder = '/kaggle/input/bkai-dataset-1/train_gt/training_gt'
BKAI1_train_output_images_folder = "/kaggle/working/train/images"
BKAI1_train_output_labels_folder = "/kaggle/working/train/labels"

In [4]:
# Total


total_test_images_folder = "/kaggle/input/totaltextstr/Total-Text/Test"
total_test_labels_folder = "/kaggle/input/totaltextstr/Total-Text/Annotation/groundtruth_polygonal_annotation/Test"

total_test_output_images_folder = "/kaggle/working/test_total/images"
total_test_output_labels_folder = "/kaggle/working/test_total/labels"

In [5]:
# ICDAR2015
icdar2015_test_images_folder = "/kaggle/input/icdar2015/ch4_test_images"
icdar2015_test_labels_folder = "/kaggle/input/icdar2015/ch4_test_localization_transcription_gt"

icdar2015_test_output_images_folder = "/kaggle/working/test_icdar2015/images"
icdar2015_test_output_labels_folder = "/kaggle/working/test_icdar2015/labels"

# Icdar2015 Dataset Test


In [6]:
def Icidar2015_process_annotations(train_images_folder, train_labels_folder, train_output_images_folder, train_output_labels_folder):
    # Tạo các thư mục đầu ra nếu chưa tồn tại
    os.makedirs(train_output_images_folder, exist_ok=True)
    os.makedirs(train_output_labels_folder, exist_ok=True)

    # Hàm chuyển đổi tọa độ polygon sang YOLO bounding box
    def polygon_to_yolo_bbox(x, y, width_img, height_img):
        x_min, x_max = min(x), max(x)
        y_min, y_max = min(y), max(y)
        x_center = (x_min + x_max) / 2 / width_img
        y_center = (y_min + y_max) / 2 / height_img
        bbox_width = (x_max - x_min) / width_img
        bbox_height = (y_max - y_min) / height_img
        return x_center, y_center, bbox_width, bbox_height

    # Duyệt qua các file annotation
    annotation_files = glob.glob(os.path.join(train_labels_folder, "*.txt"))

    for anno_file in annotation_files:
        base_name = os.path.basename(anno_file).replace("gt_", "").replace(".txt", "")
        image_path = os.path.join(train_images_folder, f"{base_name}.jpg")
        
        try:
            # Mở ảnh để lấy kích thước
            img = Image.open(image_path)
            width_img, height_img = img.size

            # Lưu ảnh vào thư mục đầu ra
            output_image_path = os.path.join(train_output_images_folder, f"{base_name}.jpg")
            
            # Biến kiểm tra xem có bounding box hợp lệ không
            valid_boxes = False
            
            # Lưu file annotation mới vào thư mục đầu ra
            output_label_file = os.path.join(train_output_labels_folder, f"{base_name}.txt")
            
            with open(anno_file, "r", encoding="utf-8-sig") as f, open(output_label_file, "w") as out_f:
                for line in f:
                    try:
                        # Loại bỏ ký tự BOM nếu có
                        line = line.lstrip('\ufeff')

                        # Tách các phần tử trong dòng annotation
                        parts = line.strip().split(',')
                        if len(parts) < 8:  # Nếu số phần tử không đủ, bỏ qua
                            continue

                        # Tách tọa độ x, y và văn bản
                        x = list(map(int, parts[:8:2]))  # Tọa độ x
                        y = list(map(int, parts[1:8:2]))  # Tọa độ y
                        transcription = parts[8]  # Văn bản

                        # # Nếu văn bản là '###', bỏ qua
                        # if transcription == '###':
                        #     continue
                        
                        # Tính toán bounding box YOLO
                        x_center, y_center, bbox_width, bbox_height = polygon_to_yolo_bbox(x, y, width_img, height_img)
                        
                        # Nếu bounding box hợp lệ, đánh dấu
                        if bbox_width > 0 and bbox_height > 0:
                            valid_boxes = True
                        
                        # Gán class_id (ví dụ: 0 cho tất cả các lớp trong OCR)
                        class_id = 0
                        
                        # Ghi vào file annotation
                        out_f.write(f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}\n")
                    except Exception as e:
                        print(f"Lỗi xử lý dòng trong file {anno_file}: {line}\nChi tiết lỗi: {e}")
            
            # Chỉ lưu ảnh và file annotation nếu có ít nhất một bounding box hợp lệ
            if valid_boxes:
                img.save(output_image_path)  # Lưu ảnh vào thư mục output
            else:
                os.remove(output_label_file)  # Xóa file label nếu không có bounding box hợp lệ
                print(f"Không có bounding box hợp lệ cho ảnh {base_name}.jpg, file annotation không được lưu.")
                
        except Exception as e:
            print(f"Lỗi xử lý file: {anno_file}\nChi tiết lỗi: {e}")

In [7]:
Icidar2015_process_annotations(icdar2015_test_images_folder, icdar2015_test_labels_folder, icdar2015_test_output_images_folder, icdar2015_test_output_labels_folder)

# Total Dataset test


In [8]:
def Total_process_annotations(train_images_folder, train_labels_folder, train_output_images_folder, train_output_labels_folder):
    # Tạo các thư mục đầu ra nếu chưa tồn tại
    os.makedirs(train_output_images_folder, exist_ok=True)
    os.makedirs(train_output_labels_folder, exist_ok=True)

    # Hàm chuyển đổi tọa độ polygon sang YOLO bounding box
    def polygon_to_yolo_bbox(x, y, width_img, height_img):
        x_min, x_max = min(x), max(x)
        y_min, y_max = min(y), max(y)
        x_center = (x_min + x_max) / 2 / width_img
        y_center = (y_min + y_max) / 2 / height_img
        bbox_width = (x_max - x_min) / width_img
        bbox_height = (y_max - y_min) / height_img
        return x_center, y_center, bbox_width, bbox_height

    # Duyệt qua các file annotation
    annotation_files = glob.glob(os.path.join(train_labels_folder, "*.txt"))

    for anno_file in annotation_files:
        base_name = os.path.basename(anno_file).replace("poly_gt_", "").replace(".txt", "")
        image_path = os.path.join(train_images_folder, f"{base_name}.jpg")
        
        try:
            # Mở ảnh để lấy kích thước
            img = Image.open(image_path)
            width_img, height_img = img.size

            # Lưu ảnh vào thư mục đầu ra
            output_image_path = os.path.join(train_output_images_folder, f"{base_name}.jpg")
            
            # Biến kiểm tra xem có bounding box hợp lệ không
            valid_boxes = False
            
            # Lưu file annotation mới vào thư mục đầu ra
            output_label_file = os.path.join(train_output_labels_folder, f"{base_name}.txt")
            
            with open(anno_file, "r") as f, open(output_label_file, "w") as out_f:
                for line in f:
                    try:
                        # Tách các phần tử trong dòng annotation và bỏ qua các phần không cần thiết
                        line = line.strip()
                        if "x: [[" not in line or "y: [[" not in line:
                            continue

                        # Lấy phần tọa độ x và y
                        x_part = line[line.index("x: [[") + 5: line.index("]], y:")]
                        y_part = line[line.index("y: [[") + 5: line.index("]], ornt:")]

                        # Chuyển đổi tọa độ x và y thành danh sách số
                        x = list(map(int, x_part.split()))
                        y = list(map(int, y_part.split()))

                        # Bỏ qua nếu số điểm không đủ
                        if len(x) < 4 or len(y) < 4:
                            continue
                        
                        # Tính toán bounding box YOLO
                        x_center, y_center, bbox_width, bbox_height = polygon_to_yolo_bbox(x, y, width_img, height_img)
                        
                        # Nếu bounding box hợp lệ, đánh dấu
                        if bbox_width > 0 and bbox_height > 0:
                            valid_boxes = True
                        
                        # Gán class_id (ví dụ: 0 cho tất cả các lớp trong OCR)
                        class_id = 0
                        
                        # Ghi vào file annotation
                        out_f.write(f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}\n")
                    except Exception as e:
                        print(f"Lỗi xử lý dòng trong file {anno_file}: {line}\nChi tiết lỗi: {e}")
            
            # Chỉ lưu ảnh và file annotation nếu có ít nhất một bounding box hợp lệ
            if valid_boxes:
                img.save(output_image_path)  # Lưu ảnh vào thư mục output
            else:
                os.remove(output_label_file)  # Xóa file label nếu không có bounding box hợp lệ
                print(f"Không có bounding box hợp lệ cho ảnh {base_name}.jpg, file annotation không được lưu.")
                
        except Exception as e:
            print(f"Lỗi xử lý file: {anno_file}\nChi tiết lỗi: {e}")

In [9]:
Total_process_annotations(total_test_images_folder, total_test_labels_folder, total_test_output_images_folder, total_test_output_labels_folder)


# BKAI DATASET TEST


In [10]:
import os
import glob
from PIL import Image

def test_process_annotations(train_images_folder, train_labels_folder, train_output_images_folder, train_output_labels_folder):
    # Tạo các thư mục đầu ra nếu chưa tồn tại
    os.makedirs(train_output_images_folder, exist_ok=True)
    os.makedirs(train_output_labels_folder, exist_ok=True)

    # Hàm chuyển đổi tọa độ polygon sang YOLO bounding box
    def polygon_to_yolo_bbox(x, y, width_img, height_img):
        x_min, x_max = min(x), max(x)
        y_min, y_max = min(y), max(y)
        x_center = (x_min + x_max) / 2 / width_img
        y_center = (y_min + y_max) / 2 / height_img
        bbox_width = (x_max - x_min) / width_img
        bbox_height = (y_max - y_min) / height_img
        return x_center, y_center, bbox_width, bbox_height

    # Duyệt qua các file annotation
    annotation_files = glob.glob(os.path.join(train_labels_folder, "*.txt"))

    for anno_file in annotation_files:
        base_name = os.path.basename(anno_file).replace("gt_", "").replace(".txt", "")
        image_path = os.path.join(train_images_folder, f"{base_name}.jpg")
        
        try:
            # Mở ảnh để lấy kích thước
            img = Image.open(image_path)
            width_img, height_img = img.size

            # Lưu ảnh vào thư mục đầu ra
            output_image_path = os.path.join(train_output_images_folder, f"{base_name}.jpg")
            
            # Biến kiểm tra xem có bounding box hợp lệ không
            valid_boxes = False
            
            # Lưu file annotation mới vào thư mục đầu ra
            output_label_file = os.path.join(train_output_labels_folder, f"{base_name}.txt")
            
            with open(anno_file, "r", encoding="utf-8-sig") as f, open(output_label_file, "w") as out_f:
                for line in f:
                    try:
                        # Loại bỏ ký tự BOM nếu có
                        line = line.lstrip('\ufeff')

                        # Tách các phần tử trong dòng annotation
                        parts = line.strip().split(',')
                        if len(parts) < 8:  # Nếu số phần tử không đủ, bỏ qua
                            continue

                        # Tách tọa độ x, y và văn bản
                        x = list(map(int, parts[:8:2]))  # Tọa độ x
                        y = list(map(int, parts[1:8:2]))  # Tọa độ y
                        transcription = parts[8]  # Văn bản

                        # Nếu bounding box không hợp lệ, bỏ qua
                        x_center, y_center, bbox_width, bbox_height = polygon_to_yolo_bbox(x, y, width_img, height_img)
                        if bbox_width > 0 and bbox_height > 0:
                            valid_boxes = True
                        
                        # Gán class_id (ví dụ: 0 cho tất cả các lớp trong OCR)
                        class_id = 0
                        
                        # Ghi vào file annotation
                        out_f.write(f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}\n")
                    except Exception as e:
                        print(f"Lỗi xử lý dòng trong file {anno_file}: {line}\nChi tiết lỗi: {e}")
            
            # Chỉ lưu ảnh và file annotation nếu có ít nhất một bounding box hợp lệ
            if valid_boxes:
                # Chuyển đổi chế độ ảnh nếu cần thiết
                if img.mode == "RGBA" or img.mode == "P":
                    img = img.convert("RGB")
                img.save(output_image_path, "JPEG",quality=95)  # Lưu ảnh vào thư mục output
            else:
                os.remove(output_label_file)  # Xóa file label nếu không có bounding box hợp lệ
                print(f"Không có bounding box hợp lệ cho ảnh {base_name}.jpg, file annotation không được lưu.")
                
        except Exception as e:
            print(f"Lỗi xử lý file: {anno_file}\nChi tiết lỗi: {e}")

In [11]:
test_images_folder = "/kaggle/input/test-bkai/private_test_imgs"
test_labels_folder = "/kaggle/input/test-bkai/groundtruth"
test_output_images_folder = "/kaggle/working/test/images"
test_output_labels_folder = "/kaggle/working/test/labels"

test_process_annotations(test_images_folder, test_labels_folder, test_output_images_folder, test_output_labels_folder)

# Data yaml

In [12]:
# Tạo đường dẫn tới file YAML
dataset_dir = "/kaggle/working"
os.makedirs(dataset_dir, exist_ok=True)
yaml_file = os.path.join(dataset_dir, "data_BKAI.yaml")

# Nội dung file YAML
yaml_content = """
train: /kaggle/input/yolov8x-ocr/train/images
val: /kaggle/working/test/images
nc: 1
names: ['text']  # Cập nhật nếu bạn có nhiều lớp

"""

# Ghi nội dung vào file YAML
with open(yaml_file, "w") as f:
    f.write(yaml_content.strip())

print(f"File 'data_BKAI.yaml' đã được tạo tại: {yaml_file}")

File 'data_BKAI.yaml' đã được tạo tại: /kaggle/working/data_BKAI.yaml


In [13]:
# Tạo đường dẫn tới file YAML
dataset_dir = "/kaggle/working"
os.makedirs(dataset_dir, exist_ok=True)
yaml_file = os.path.join(dataset_dir, "data_Total.yaml")

# Nội dung file YAML
yaml_content = """
train: /kaggle/input/yolov8x-ocr/train/images
val: /kaggle/working/test_total/images
nc: 1
names: ['text']  # Cập nhật nếu bạn có nhiều lớp

"""

# Ghi nội dung vào file YAML
with open(yaml_file, "w") as f:
    f.write(yaml_content.strip())

print(f"File 'data_Total.yaml' đã được tạo tại: {yaml_file}")

File 'data_Total.yaml' đã được tạo tại: /kaggle/working/data_Total.yaml


In [14]:
# Tạo đường dẫn tới file YAML
dataset_dir = "/kaggle/working"
os.makedirs(dataset_dir, exist_ok=True)
yaml_file = os.path.join(dataset_dir, "data_Icdar2015.yaml")

# Nội dung file YAML
yaml_content = """
train: /kaggle/input/yolov8x-ocr/train/images
val: /kaggle/working/test_icdar2015/images
nc: 1
names: ['text']  # Cập nhật nếu bạn có nhiều lớp

"""

# Ghi nội dung vào file YAML
with open(yaml_file, "w") as f:
    f.write(yaml_content.strip())

print(f"File 'data_Icdar2015.yaml' đã được tạo tại: {yaml_file}")

File 'data_Icdar2015.yaml' đã được tạo tại: /kaggle/working/data_Icdar2015.yaml


# Train lai - Step 1 - SGD ver 1



In [15]:
from ultralytics import YOLO
import os
import cv2

# Đường dẫn thư mục chứa các ảnh và mô hình YOLOv8
image_path = '/kaggle/input/test-bkai/private_test_imgs'
model_path = '/kaggle/input/train-lai-step-1-vinhx-ver-1/pytorch/default/1/best (9).pt'

# Load mô hình YOLOv8 từ đường dẫn
model = YOLO(model_path)
results = model.val(data='/kaggle/working/data_BKAI.yaml', imgsz=640)  # Đảm bảo rằng file data.yaml đúng với cấu trúc dữ liệu của bạn

# Lấy các chỉ số từ kết quả
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP50 = results.results_dict['metrics/mAP50(B)']
mAP50_95 = results.results_dict['metrics/mAP50-95(B)']

# Tính F1-Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Hiển thị các chỉ số
print("Overall Metrics:")
print(f"Average Precision: {precision:.4f}")
print(f"Average Recall: {recall:.4f}")
print(f"Average F1-Score: {f1_score:.4f}")
print(f"mAP50: {mAP50:.4f}")
print(f"mAP50-95: {mAP50_95:.4f}")

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 268 layers, 68,124,531 parameters, 0 gradients, 257.4 GFLOPs


100%|██████████| 755k/755k [00:00<00:00, 26.1MB/s]
val: Scanning /kaggle/working/test/labels... 300 images, 0 backgrounds, 1 corrupt: 100%|██████████| 300/300 [00:00<00:00, 1246.88it/s]

val: WARNING ⚠️ /kaggle/working/test/images/img_238.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.1983]
val: New cache created: /kaggle/working/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:07<00:00,  2.45it/s]


                   all        299        866      0.841      0.688       0.76      0.416
Speed: 0.4ms preprocess, 16.7ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs/detect/val
Overall Metrics:
Average Precision: 0.8413
Average Recall: 0.6882
Average F1-Score: 0.7571
mAP50: 0.7603
mAP50-95: 0.4157


In [16]:
results = model.val(data='/kaggle/working/data_Icdar2015.yaml', imgsz=640)  # Đảm bảo rằng file data.yaml đúng với cấu trúc dữ liệu của bạn

# Lấy các chỉ số từ kết quả
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP50 = results.results_dict['metrics/mAP50(B)']
mAP50_95 = results.results_dict['metrics/mAP50-95(B)']

# Tính F1-Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Hiển thị các chỉ số
print("Overall Metrics:")
print(f"Average Precision: {precision:.4f}")
print(f"Average Recall: {recall:.4f}")
print(f"Average F1-Score: {f1_score:.4f}")
print(f"mAP50: {mAP50:.4f}")
print(f"mAP50-95: {mAP50_95:.4f}")

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)


val: Scanning /kaggle/working/test_icdar2015/labels... 500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<00:00, 1110.45it/s]

val: New cache created: /kaggle/working/test_icdar2015/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:11<00:00,  2.80it/s]


                   all        500       5230      0.685      0.525      0.589      0.318
Speed: 0.5ms preprocess, 16.1ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to runs/detect/val2
Overall Metrics:
Average Precision: 0.6851
Average Recall: 0.5252
Average F1-Score: 0.5946
mAP50: 0.5894
mAP50-95: 0.3185


In [17]:
results = model.val(data='/kaggle/working/data_Total.yaml', imgsz=640)  # Đảm bảo rằng file data.yaml đúng với cấu trúc dữ liệu của bạn

# Lấy các chỉ số từ kết quả
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP50 = results.results_dict['metrics/mAP50(B)']
mAP50_95 = results.results_dict['metrics/mAP50-95(B)']

# Tính F1-Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Hiển thị các chỉ số
print("Overall Metrics:")
print(f"Average Precision: {precision:.4f}")
print(f"Average Recall: {recall:.4f}")
print(f"Average F1-Score: {f1_score:.4f}")
print(f"mAP50: {mAP50:.4f}")
print(f"mAP50-95: {mAP50_95:.4f}")

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)


val: Scanning /kaggle/working/test_total/labels... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<00:00, 1173.12it/s]

val: New cache created: /kaggle/working/test_total/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:09<00:00,  2.09it/s]


                   all        300       2540       0.82      0.734      0.811      0.542
Speed: 0.8ms preprocess, 21.2ms inference, 0.0ms loss, 3.2ms postprocess per image
Results saved to runs/detect/val3
Overall Metrics:
Average Precision: 0.8201
Average Recall: 0.7340
Average F1-Score: 0.7746
mAP50: 0.8106
mAP50-95: 0.5421


# Train lai - Step 1 - SGD ver 2



In [18]:
from ultralytics import YOLO
import os
import cv2

# Đường dẫn thư mục chứa các ảnh và mô hình YOLOv8
image_path = '/kaggle/input/test-bkai/private_test_imgs'
model_path = '/kaggle/input/train-lai-step-1-vinhx-ver-2/pytorch/default/1/best.pt'

# Load mô hình YOLOv8 từ đường dẫn
model = YOLO(model_path)
results = model.val(data='/kaggle/working/data_BKAI.yaml', imgsz=640)  # Đảm bảo rằng file data.yaml đúng với cấu trúc dữ liệu của bạn

# Lấy các chỉ số từ kết quả
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP50 = results.results_dict['metrics/mAP50(B)']
mAP50_95 = results.results_dict['metrics/mAP50-95(B)']

# Tính F1-Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Hiển thị các chỉ số
print("Overall Metrics:")
print(f"Average Precision: {precision:.4f}")
print(f"Average Recall: {recall:.4f}")
print(f"Average F1-Score: {f1_score:.4f}")
print(f"mAP50: {mAP50:.4f}")
print(f"mAP50-95: {mAP50_95:.4f}")

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 268 layers, 68,124,531 parameters, 0 gradients, 257.4 GFLOPs


val: Scanning /kaggle/working/test/labels.cache... 300 images, 0 backgrounds, 1 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]

val: WARNING ⚠️ /kaggle/working/test/images/img_238.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.1983]



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:06<00:00,  2.98it/s]


                   all        299        866       0.88      0.693      0.787      0.452
Speed: 0.5ms preprocess, 16.7ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/val4
Overall Metrics:
Average Precision: 0.8797
Average Recall: 0.6928
Average F1-Score: 0.7752
mAP50: 0.7866
mAP50-95: 0.4517


In [19]:
results = model.val(data='/kaggle/working/data_Icdar2015.yaml', imgsz=640)  # Đảm bảo rằng file data.yaml đúng với cấu trúc dữ liệu của bạn

# Lấy các chỉ số từ kết quả
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP50 = results.results_dict['metrics/mAP50(B)']
mAP50_95 = results.results_dict['metrics/mAP50-95(B)']

# Tính F1-Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Hiển thị các chỉ số
print("Overall Metrics:")
print(f"Average Precision: {precision:.4f}")
print(f"Average Recall: {recall:.4f}")
print(f"Average F1-Score: {f1_score:.4f}")
print(f"mAP50: {mAP50:.4f}")
print(f"mAP50-95: {mAP50_95:.4f}")

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)


val: Scanning /kaggle/working/test_icdar2015/labels.cache... 500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:11<00:00,  2.81it/s]


                   all        500       5230      0.685      0.551      0.613      0.352
Speed: 0.4ms preprocess, 15.6ms inference, 0.1ms loss, 2.3ms postprocess per image
Results saved to runs/detect/val5
Overall Metrics:
Average Precision: 0.6849
Average Recall: 0.5511
Average F1-Score: 0.6107
mAP50: 0.6128
mAP50-95: 0.3518


In [20]:
results = model.val(data='/kaggle/working/data_Total.yaml', imgsz=640)  # Đảm bảo rằng file data.yaml đúng với cấu trúc dữ liệu của bạn

# Lấy các chỉ số từ kết quả
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP50 = results.results_dict['metrics/mAP50(B)']
mAP50_95 = results.results_dict['metrics/mAP50-95(B)']

# Tính F1-Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Hiển thị các chỉ số
print("Overall Metrics:")
print(f"Average Precision: {precision:.4f}")
print(f"Average Recall: {recall:.4f}")
print(f"Average F1-Score: {f1_score:.4f}")
print(f"mAP50: {mAP50:.4f}")
print(f"mAP50-95: {mAP50_95:.4f}")

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)


val: Scanning /kaggle/working/test_total/labels.cache... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:08<00:00,  2.15it/s]


                   all        300       2540      0.819      0.762      0.816      0.565
Speed: 1.0ms preprocess, 21.9ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs/detect/val6
Overall Metrics:
Average Precision: 0.8191
Average Recall: 0.7618
Average F1-Score: 0.7894
mAP50: 0.8160
mAP50-95: 0.5649


# Train lai - Step 1 - SGD ver 3


In [21]:
from ultralytics import YOLO
import os
import cv2

# Đường dẫn thư mục chứa các ảnh và mô hình YOLOv8
image_path = '/kaggle/input/test-bkai/private_test_imgs'
model_path = '/kaggle/input/step-1-vinhx-ver-3/pytorch/default/1/best.pt'

# Load mô hình YOLOv8 từ đường dẫn
model = YOLO(model_path)
results = model.val(data='/kaggle/working/data_BKAI.yaml', imgsz=640)  # Đảm bảo rằng file data.yaml đúng với cấu trúc dữ liệu của bạn

# Lấy các chỉ số từ kết quả
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP50 = results.results_dict['metrics/mAP50(B)']
mAP50_95 = results.results_dict['metrics/mAP50-95(B)']

# Tính F1-Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Hiển thị các chỉ số
print("Overall Metrics:")
print(f"Average Precision: {precision:.4f}")
print(f"Average Recall: {recall:.4f}")
print(f"Average F1-Score: {f1_score:.4f}")
print(f"mAP50: {mAP50:.4f}")
print(f"mAP50-95: {mAP50_95:.4f}")

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 268 layers, 68,124,531 parameters, 0 gradients, 257.4 GFLOPs


val: Scanning /kaggle/working/test/labels.cache... 300 images, 0 backgrounds, 1 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]

val: WARNING ⚠️ /kaggle/working/test/images/img_238.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.1983]



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:06<00:00,  3.02it/s]


                   all        299        866      0.834      0.711      0.799      0.431
Speed: 0.7ms preprocess, 16.3ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/val7
Overall Metrics:
Average Precision: 0.8335
Average Recall: 0.7110
Average F1-Score: 0.7674
mAP50: 0.7995
mAP50-95: 0.4307


In [22]:
results = model.val(data='/kaggle/working/data_Icdar2015.yaml', imgsz=640)  # Đảm bảo rằng file data.yaml đúng với cấu trúc dữ liệu của bạn

# Lấy các chỉ số từ kết quả
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP50 = results.results_dict['metrics/mAP50(B)']
mAP50_95 = results.results_dict['metrics/mAP50-95(B)']

# Tính F1-Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Hiển thị các chỉ số
print("Overall Metrics:")
print(f"Average Precision: {precision:.4f}")
print(f"Average Recall: {recall:.4f}")
print(f"Average F1-Score: {f1_score:.4f}")
print(f"mAP50: {mAP50:.4f}")
print(f"mAP50-95: {mAP50_95:.4f}")

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)


val: Scanning /kaggle/working/test_icdar2015/labels.cache... 500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:11<00:00,  2.76it/s]


                   all        500       5230      0.682      0.559       0.61      0.334
Speed: 0.5ms preprocess, 15.8ms inference, 0.0ms loss, 2.2ms postprocess per image
Results saved to runs/detect/val8
Overall Metrics:
Average Precision: 0.6816
Average Recall: 0.5591
Average F1-Score: 0.6143
mAP50: 0.6104
mAP50-95: 0.3345


In [23]:
results = model.val(data='/kaggle/working/data_Total.yaml', imgsz=640)  # Đảm bảo rằng file data.yaml đúng với cấu trúc dữ liệu của bạn

# Lấy các chỉ số từ kết quả
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP50 = results.results_dict['metrics/mAP50(B)']
mAP50_95 = results.results_dict['metrics/mAP50-95(B)']

# Tính F1-Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Hiển thị các chỉ số
print("Overall Metrics:")
print(f"Average Precision: {precision:.4f}")
print(f"Average Recall: {recall:.4f}")
print(f"Average F1-Score: {f1_score:.4f}")
print(f"mAP50: {mAP50:.4f}")
print(f"mAP50-95: {mAP50_95:.4f}")

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)


val: Scanning /kaggle/working/test_total/labels.cache... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:09<00:00,  2.08it/s]


                   all        300       2540       0.82      0.752      0.816      0.555
Speed: 0.8ms preprocess, 21.4ms inference, 0.0ms loss, 3.3ms postprocess per image
Results saved to runs/detect/val9
Overall Metrics:
Average Precision: 0.8197
Average Recall: 0.7519
Average F1-Score: 0.7844
mAP50: 0.8157
mAP50-95: 0.5546


# Train lai - Step 1 - SGD ver 4


In [24]:
from ultralytics import YOLO
import os
import cv2

# Đường dẫn thư mục chứa các ảnh và mô hình YOLOv8
image_path = '/kaggle/input/test-bkai/private_test_imgs'
model_path = '/kaggle/input/train-lai-step-1-vinhx-ver-4/pytorch/default/1/best (1).pt'

# Load mô hình YOLOv8 từ đường dẫn
model = YOLO(model_path)
results = model.val(data='/kaggle/working/data_BKAI.yaml', imgsz=640)  # Đảm bảo rằng file data.yaml đúng với cấu trúc dữ liệu của bạn

# Lấy các chỉ số từ kết quả
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP50 = results.results_dict['metrics/mAP50(B)']
mAP50_95 = results.results_dict['metrics/mAP50-95(B)']

# Tính F1-Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Hiển thị các chỉ số
print("Overall Metrics:")
print(f"Average Precision: {precision:.4f}")
print(f"Average Recall: {recall:.4f}")
print(f"Average F1-Score: {f1_score:.4f}")
print(f"mAP50: {mAP50:.4f}")
print(f"mAP50-95: {mAP50_95:.4f}")

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 268 layers, 68,124,531 parameters, 0 gradients, 257.4 GFLOPs


val: Scanning /kaggle/working/test/labels.cache... 300 images, 0 backgrounds, 1 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]

val: WARNING ⚠️ /kaggle/working/test/images/img_238.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.1983]



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:06<00:00,  3.05it/s]


                   all        299        866      0.868      0.706      0.792      0.419
Speed: 0.4ms preprocess, 15.6ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs/detect/val10
Overall Metrics:
Average Precision: 0.8679
Average Recall: 0.7056
Average F1-Score: 0.7784
mAP50: 0.7922
mAP50-95: 0.4193


In [25]:
results = model.val(data='/kaggle/working/data_Icdar2015.yaml', imgsz=640)  # Đảm bảo rằng file data.yaml đúng với cấu trúc dữ liệu của bạn

# Lấy các chỉ số từ kết quả
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP50 = results.results_dict['metrics/mAP50(B)']
mAP50_95 = results.results_dict['metrics/mAP50-95(B)']

# Tính F1-Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Hiển thị các chỉ số
print("Overall Metrics:")
print(f"Average Precision: {precision:.4f}")
print(f"Average Recall: {recall:.4f}")
print(f"Average F1-Score: {f1_score:.4f}")
print(f"mAP50: {mAP50:.4f}")
print(f"mAP50-95: {mAP50_95:.4f}")

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)


val: Scanning /kaggle/working/test_icdar2015/labels.cache... 500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:11<00:00,  2.81it/s]


                   all        500       5230      0.689      0.543      0.598      0.327
Speed: 0.4ms preprocess, 16.2ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs/detect/val11
Overall Metrics:
Average Precision: 0.6887
Average Recall: 0.5427
Average F1-Score: 0.6070
mAP50: 0.5982
mAP50-95: 0.3272


In [26]:
results = model.val(data='/kaggle/working/data_Total.yaml', imgsz=640)  # Đảm bảo rằng file data.yaml đúng với cấu trúc dữ liệu của bạn

# Lấy các chỉ số từ kết quả
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP50 = results.results_dict['metrics/mAP50(B)']
mAP50_95 = results.results_dict['metrics/mAP50-95(B)']

# Tính F1-Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Hiển thị các chỉ số
print("Overall Metrics:")
print(f"Average Precision: {precision:.4f}")
print(f"Average Recall: {recall:.4f}")
print(f"Average F1-Score: {f1_score:.4f}")
print(f"mAP50: {mAP50:.4f}")
print(f"mAP50-95: {mAP50_95:.4f}")

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)


val: Scanning /kaggle/working/test_total/labels.cache... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:09<00:00,  2.08it/s]


                   all        300       2540      0.825      0.728      0.804       0.54
Speed: 0.7ms preprocess, 20.2ms inference, 0.0ms loss, 3.3ms postprocess per image
Results saved to runs/detect/val12
Overall Metrics:
Average Precision: 0.8250
Average Recall: 0.7277
Average F1-Score: 0.7733
mAP50: 0.8043
mAP50-95: 0.5399


# Train lai - Step 1 - AdamW ver 1


In [27]:
from ultralytics import YOLO
import os
import cv2

# Đường dẫn thư mục chứa các ảnh và mô hình YOLOv8
image_path = '/kaggle/input/test-bkai/private_test_imgs'
model_path = '/kaggle/input/train-again-step-1-uit-ver-1/pytorch/default/1/best (9).pt'

# Load mô hình YOLOv8 từ đường dẫn
model = YOLO(model_path)
results = model.val(data='/kaggle/working/data_BKAI.yaml', imgsz=640)  # Đảm bảo rằng file data.yaml đúng với cấu trúc dữ liệu của bạn

# Lấy các chỉ số từ kết quả
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP50 = results.results_dict['metrics/mAP50(B)']
mAP50_95 = results.results_dict['metrics/mAP50-95(B)']

# Tính F1-Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Hiển thị các chỉ số
print("Overall Metrics:")
print(f"Average Precision: {precision:.4f}")
print(f"Average Recall: {recall:.4f}")
print(f"Average F1-Score: {f1_score:.4f}")
print(f"mAP50: {mAP50:.4f}")
print(f"mAP50-95: {mAP50_95:.4f}")

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 268 layers, 68,124,531 parameters, 0 gradients, 257.4 GFLOPs


val: Scanning /kaggle/working/test/labels.cache... 300 images, 0 backgrounds, 1 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]

val: WARNING ⚠️ /kaggle/working/test/images/img_238.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.1983]



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:06<00:00,  3.09it/s]


                   all        299        866      0.788      0.594      0.677       0.33
Speed: 0.8ms preprocess, 15.7ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/val13
Overall Metrics:
Average Precision: 0.7883
Average Recall: 0.5935
Average F1-Score: 0.6772
mAP50: 0.6766
mAP50-95: 0.3298


In [28]:
results = model.val(data='/kaggle/working/data_Icdar2015.yaml', imgsz=640)  # Đảm bảo rằng file data.yaml đúng với cấu trúc dữ liệu của bạn

# Lấy các chỉ số từ kết quả
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP50 = results.results_dict['metrics/mAP50(B)']
mAP50_95 = results.results_dict['metrics/mAP50-95(B)']

# Tính F1-Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Hiển thị các chỉ số
print("Overall Metrics:")
print(f"Average Precision: {precision:.4f}")
print(f"Average Recall: {recall:.4f}")
print(f"Average F1-Score: {f1_score:.4f}")
print(f"mAP50: {mAP50:.4f}")
print(f"mAP50-95: {mAP50_95:.4f}")

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)


val: Scanning /kaggle/working/test_icdar2015/labels.cache... 500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:11<00:00,  2.79it/s]


                   all        500       5230      0.701      0.533      0.592      0.321
Speed: 0.7ms preprocess, 16.8ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/val14
Overall Metrics:
Average Precision: 0.7005
Average Recall: 0.5327
Average F1-Score: 0.6052
mAP50: 0.5918
mAP50-95: 0.3213


In [29]:
results = model.val(data='/kaggle/working/data_Total.yaml', imgsz=640)  # Đảm bảo rằng file data.yaml đúng với cấu trúc dữ liệu của bạn

# Lấy các chỉ số từ kết quả
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP50 = results.results_dict['metrics/mAP50(B)']
mAP50_95 = results.results_dict['metrics/mAP50-95(B)']

# Tính F1-Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Hiển thị các chỉ số
print("Overall Metrics:")
print(f"Average Precision: {precision:.4f}")
print(f"Average Recall: {recall:.4f}")
print(f"Average F1-Score: {f1_score:.4f}")
print(f"mAP50: {mAP50:.4f}")
print(f"mAP50-95: {mAP50_95:.4f}")

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)


val: Scanning /kaggle/working/test_total/labels.cache... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:09<00:00,  2.07it/s]


                   all        300       2540      0.823      0.726        0.8      0.522
Speed: 1.2ms preprocess, 20.9ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/val15
Overall Metrics:
Average Precision: 0.8234
Average Recall: 0.7256
Average F1-Score: 0.7714
mAP50: 0.8001
mAP50-95: 0.5222


# Train lai - Step 1 - AdamW ver 2


In [30]:
from ultralytics import YOLO
import os
import cv2

# Đường dẫn thư mục chứa các ảnh và mô hình YOLOv8
image_path = '/kaggle/input/test-bkai/private_test_imgs'
model_path = '/kaggle/input/train-lai-uit-ver-2/pytorch/default/1/best.pt'

# Load mô hình YOLOv8 từ đường dẫn
model = YOLO(model_path)
results = model.val(data='/kaggle/working/data_BKAI.yaml', imgsz=640)  # Đảm bảo rằng file data.yaml đúng với cấu trúc dữ liệu của bạn

# Lấy các chỉ số từ kết quả
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP50 = results.results_dict['metrics/mAP50(B)']
mAP50_95 = results.results_dict['metrics/mAP50-95(B)']

# Tính F1-Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Hiển thị các chỉ số
print("Overall Metrics:")
print(f"Average Precision: {precision:.4f}")
print(f"Average Recall: {recall:.4f}")
print(f"Average F1-Score: {f1_score:.4f}")
print(f"mAP50: {mAP50:.4f}")
print(f"mAP50-95: {mAP50_95:.4f}")

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 268 layers, 68,124,531 parameters, 0 gradients, 257.4 GFLOPs


val: Scanning /kaggle/working/test/labels.cache... 300 images, 0 backgrounds, 1 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]

val: WARNING ⚠️ /kaggle/working/test/images/img_238.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.1983]



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:06<00:00,  3.12it/s]


                   all        299        866      0.863      0.694      0.769      0.423
Speed: 0.4ms preprocess, 15.9ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/val16
Overall Metrics:
Average Precision: 0.8635
Average Recall: 0.6939
Average F1-Score: 0.7695
mAP50: 0.7690
mAP50-95: 0.4231


In [31]:
results = model.val(data='/kaggle/working/data_Icdar2015.yaml', imgsz=640)  # Đảm bảo rằng file data.yaml đúng với cấu trúc dữ liệu của bạn

# Lấy các chỉ số từ kết quả
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP50 = results.results_dict['metrics/mAP50(B)']
mAP50_95 = results.results_dict['metrics/mAP50-95(B)']

# Tính F1-Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Hiển thị các chỉ số
print("Overall Metrics:")
print(f"Average Precision: {precision:.4f}")
print(f"Average Recall: {recall:.4f}")
print(f"Average F1-Score: {f1_score:.4f}")
print(f"mAP50: {mAP50:.4f}")
print(f"mAP50-95: {mAP50_95:.4f}")

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)


val: Scanning /kaggle/working/test_icdar2015/labels.cache... 500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:11<00:00,  2.77it/s]


                   all        500       5230      0.699      0.561      0.607      0.334
Speed: 0.5ms preprocess, 15.6ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to runs/detect/val17
Overall Metrics:
Average Precision: 0.6990
Average Recall: 0.5614
Average F1-Score: 0.6227
mAP50: 0.6074
mAP50-95: 0.3340


In [32]:
results = model.val(data='/kaggle/working/data_Total.yaml', imgsz=640)  # Đảm bảo rằng file data.yaml đúng với cấu trúc dữ liệu của bạn

# Lấy các chỉ số từ kết quả
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP50 = results.results_dict['metrics/mAP50(B)']
mAP50_95 = results.results_dict['metrics/mAP50-95(B)']

# Tính F1-Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Hiển thị các chỉ số
print("Overall Metrics:")
print(f"Average Precision: {precision:.4f}")
print(f"Average Recall: {recall:.4f}")
print(f"Average F1-Score: {f1_score:.4f}")
print(f"mAP50: {mAP50:.4f}")
print(f"mAP50-95: {mAP50_95:.4f}")

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)


val: Scanning /kaggle/working/test_total/labels.cache... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:08<00:00,  2.14it/s]


                   all        300       2540       0.84      0.755      0.814      0.545
Speed: 1.0ms preprocess, 21.4ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/val18
Overall Metrics:
Average Precision: 0.8405
Average Recall: 0.7547
Average F1-Score: 0.7953
mAP50: 0.8141
mAP50-95: 0.5451


# Train lai - Step 1 - AdamW ver 3


In [33]:
from ultralytics import YOLO
import os
import cv2

# Đường dẫn thư mục chứa các ảnh và mô hình YOLOv8
image_path = '/kaggle/input/test-bkai/private_test_imgs'
model_path = '/kaggle/input/step-1-uit-ver-3/pytorch/default/1/best.pt'

# Load mô hình YOLOv8 từ đường dẫn
model = YOLO(model_path)
results = model.val(data='/kaggle/working/data_BKAI.yaml', imgsz=640)  # Đảm bảo rằng file data.yaml đúng với cấu trúc dữ liệu của bạn

# Lấy các chỉ số từ kết quả
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP50 = results.results_dict['metrics/mAP50(B)']
mAP50_95 = results.results_dict['metrics/mAP50-95(B)']

# Tính F1-Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Hiển thị các chỉ số
print("Overall Metrics:")
print(f"Average Precision: {precision:.4f}")
print(f"Average Recall: {recall:.4f}")
print(f"Average F1-Score: {f1_score:.4f}")
print(f"mAP50: {mAP50:.4f}")
print(f"mAP50-95: {mAP50_95:.4f}")

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 268 layers, 68,124,531 parameters, 0 gradients, 257.4 GFLOPs


val: Scanning /kaggle/working/test/labels.cache... 300 images, 0 backgrounds, 1 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]

val: WARNING ⚠️ /kaggle/working/test/images/img_238.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.1983]



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:06<00:00,  2.98it/s]


                   all        299        866      0.826      0.617      0.715      0.358
Speed: 0.3ms preprocess, 15.8ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs/detect/val19
Overall Metrics:
Average Precision: 0.8261
Average Recall: 0.6166
Average F1-Score: 0.7061
mAP50: 0.7150
mAP50-95: 0.3580


In [34]:
results = model.val(data='/kaggle/working/data_Icdar2015.yaml', imgsz=640)  # Đảm bảo rằng file data.yaml đúng với cấu trúc dữ liệu của bạn

# Lấy các chỉ số từ kết quả
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP50 = results.results_dict['metrics/mAP50(B)']
mAP50_95 = results.results_dict['metrics/mAP50-95(B)']

# Tính F1-Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Hiển thị các chỉ số
print("Overall Metrics:")
print(f"Average Precision: {precision:.4f}")
print(f"Average Recall: {recall:.4f}")
print(f"Average F1-Score: {f1_score:.4f}")
print(f"mAP50: {mAP50:.4f}")
print(f"mAP50-95: {mAP50_95:.4f}")

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)


val: Scanning /kaggle/working/test_icdar2015/labels.cache... 500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:11<00:00,  2.67it/s]


                   all        500       5230      0.704      0.541      0.608      0.332
Speed: 0.5ms preprocess, 15.7ms inference, 0.0ms loss, 3.1ms postprocess per image
Results saved to runs/detect/val20
Overall Metrics:
Average Precision: 0.7045
Average Recall: 0.5405
Average F1-Score: 0.6117
mAP50: 0.6082
mAP50-95: 0.3323


In [35]:
results = model.val(data='/kaggle/working/data_Total.yaml', imgsz=640)  # Đảm bảo rằng file data.yaml đúng với cấu trúc dữ liệu của bạn

# Lấy các chỉ số từ kết quả
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP50 = results.results_dict['metrics/mAP50(B)']
mAP50_95 = results.results_dict['metrics/mAP50-95(B)']

# Tính F1-Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Hiển thị các chỉ số
print("Overall Metrics:")
print(f"Average Precision: {precision:.4f}")
print(f"Average Recall: {recall:.4f}")
print(f"Average F1-Score: {f1_score:.4f}")
print(f"mAP50: {mAP50:.4f}")
print(f"mAP50-95: {mAP50_95:.4f}")

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)


val: Scanning /kaggle/working/test_total/labels.cache... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:09<00:00,  2.10it/s]


                   all        300       2540      0.829      0.744      0.816      0.539
Speed: 0.9ms preprocess, 21.2ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs/detect/val21
Overall Metrics:
Average Precision: 0.8286
Average Recall: 0.7441
Average F1-Score: 0.7841
mAP50: 0.8157
mAP50-95: 0.5389


# Train lai - Step 1 - AdamW ver 4


In [36]:
from ultralytics import YOLO
import os
import cv2

# Đường dẫn thư mục chứa các ảnh và mô hình YOLOv8
image_path = '/kaggle/input/test-bkai/private_test_imgs'
model_path = '/kaggle/input/train-lai-step-1-uit-ver-4/pytorch/default/1/best.pt'

# Load mô hình YOLOv8 từ đường dẫn
model = YOLO(model_path)
results = model.val(data='/kaggle/working/data_BKAI.yaml', imgsz=640)  # Đảm bảo rằng file data.yaml đúng với cấu trúc dữ liệu của bạn

# Lấy các chỉ số từ kết quả
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP50 = results.results_dict['metrics/mAP50(B)']
mAP50_95 = results.results_dict['metrics/mAP50-95(B)']

# Tính F1-Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Hiển thị các chỉ số
print("Overall Metrics:")
print(f"Average Precision: {precision:.4f}")
print(f"Average Recall: {recall:.4f}")
print(f"Average F1-Score: {f1_score:.4f}")
print(f"mAP50: {mAP50:.4f}")
print(f"mAP50-95: {mAP50_95:.4f}")

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 268 layers, 68,124,531 parameters, 0 gradients, 257.4 GFLOPs


val: Scanning /kaggle/working/test/labels.cache... 300 images, 0 backgrounds, 1 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]

val: WARNING ⚠️ /kaggle/working/test/images/img_238.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.1983]



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:06<00:00,  3.06it/s]


                   all        299        866      0.824      0.663      0.724      0.361
Speed: 0.5ms preprocess, 15.6ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs/detect/val22
Overall Metrics:
Average Precision: 0.8237
Average Recall: 0.6628
Average F1-Score: 0.7346
mAP50: 0.7239
mAP50-95: 0.3610


In [37]:
results = model.val(data='/kaggle/working/data_Icdar2015.yaml', imgsz=640)  # Đảm bảo rằng file data.yaml đúng với cấu trúc dữ liệu của bạn

# Lấy các chỉ số từ kết quả
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP50 = results.results_dict['metrics/mAP50(B)']
mAP50_95 = results.results_dict['metrics/mAP50-95(B)']

# Tính F1-Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Hiển thị các chỉ số
print("Overall Metrics:")
print(f"Average Precision: {precision:.4f}")
print(f"Average Recall: {recall:.4f}")
print(f"Average F1-Score: {f1_score:.4f}")
print(f"mAP50: {mAP50:.4f}")
print(f"mAP50-95: {mAP50_95:.4f}")

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)


val: Scanning /kaggle/working/test_icdar2015/labels.cache... 500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:11<00:00,  2.79it/s]


                   all        500       5230       0.71      0.554      0.615      0.339
Speed: 0.6ms preprocess, 16.4ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs/detect/val23
Overall Metrics:
Average Precision: 0.7101
Average Recall: 0.5545
Average F1-Score: 0.6227
mAP50: 0.6150
mAP50-95: 0.3392


In [38]:
results = model.val(data='/kaggle/working/data_Total.yaml', imgsz=640)  # Đảm bảo rằng file data.yaml đúng với cấu trúc dữ liệu của bạn

# Lấy các chỉ số từ kết quả
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP50 = results.results_dict['metrics/mAP50(B)']
mAP50_95 = results.results_dict['metrics/mAP50-95(B)']

# Tính F1-Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Hiển thị các chỉ số
print("Overall Metrics:")
print(f"Average Precision: {precision:.4f}")
print(f"Average Recall: {recall:.4f}")
print(f"Average F1-Score: {f1_score:.4f}")
print(f"mAP50: {mAP50:.4f}")
print(f"mAP50-95: {mAP50_95:.4f}")

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)


val: Scanning /kaggle/working/test_total/labels.cache... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:09<00:00,  2.03it/s]


                   all        300       2540       0.83      0.757      0.816      0.546
Speed: 0.8ms preprocess, 20.4ms inference, 0.0ms loss, 3.5ms postprocess per image
Results saved to runs/detect/val24
Overall Metrics:
Average Precision: 0.8298
Average Recall: 0.7567
Average F1-Score: 0.7916
mAP50: 0.8160
mAP50-95: 0.5458
